In [2]:
import sys

sys.path.append('/Users/antonbaleatolizancos/Projects/direct_SHT/direct_sht/sht')
from sht import DirectSHT

import numpy as np
import numba as nb
import matplotlib.pyplot as plt
import jax.numpy as jnp
from jax import jit, vmap, device_put
import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# We can very quickly calculate Ylm values. As an example, compute them up to lmax=Nl
Nl  = 500
Nx  = 1024

sht = DirectSHT(Nl,Nx)
x_samples = sht.x

In [4]:
def get_points(Nrandoms):
    # A function to randomly generate data points
    thetas = np.random.uniform(np.pi/4, np.pi/2., Nrandoms)
    phis = np.random.uniform(0, np.pi, Nrandoms)
    wts = np.random.uniform(1, 1.5, Nrandoms)
    
    x=np.cos(thetas)

    sorted_idx = np.argsort(x)
    x_data_sorted = x[sorted_idx]; w_i_sorted = wts[:][sorted_idx]; phi_data_sorted = phis[:][sorted_idx]
    return x_data_sorted, w_i_sorted, phi_data_sorted

In [5]:
Nrandoms = int(1e7)
x_data_sorted, w_i_sorted, phi_data_sorted = get_points(Nrandoms)
spline_idx = np.digitize(x_data_sorted, x_samples) - 1
t = x_data_sorted - x_samples[spline_idx]

# Put things in the GPU
x_data_sorted_jax = device_put(x_data_sorted)
spline_idx_jax = device_put(spline_idx)

Metal device set to: Apple M2 Max


2023-11-22 20:59:50.249816: W pjrt_plugin/src/mps_client.cc:534] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!


In [27]:
from scipy.stats import mode

def find_transitions(arr):
    # Find the differences between consecutive elements
    differences = np.diff(arr)
    # Find the indices where differences are non-zero
    transition_indices = np.nonzero(differences)[0] + 1
    # Prepend a zero for convenience
    transition_indices = np.insert(transition_indices, 0, 0, axis=0)
    return transition_indices

def reshape_array(data, transitions,spline_idx, bin_num, bin_len):
    # Reshape the data into a 2D array
    data_reshaped = np.zeros((bin_num, bin_len))
    for i in range(bin_num-1):
        fill_in = data[transitions[i]:transitions[i+1]]
        data_reshaped[i,:len(fill_in)] = fill_in
    return data_reshaped

# First, we find the number of different bins that are populated
bin_num = len(np.unique(spline_idx))
# Then, we find the maximum number of points in a bin
bin_len = mode(spline_idx).count
# Find transitions
transitions = find_transitions(spline_idx)
reshaped_inputs = [device_put(reshape_array(w_i_sorted*input_,transitions,spline_idx,bin_num,bin_len)) for input_ in [(2*t+1)*(1-t)**2, t*(1-t)**2, t**2*(3-2*t), t**2*(t-1)]]

